In [9]:
# %pip install pytesseract
# %pip install unstructured-pytesseract
# %pip install faiss-cpu
from openai import OpenAI
import os
# Set up your OpenAI API key
from dotenv import load_dotenv, dotenv_values 
load_dotenv() 



openai_api_key = os.getenv("open_ai_key")

client = OpenAI(
    api_key = openai_api_key
)
    
models = client.models.list()

models


SyncPage[Model](data=[Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', own

In [1]:
import os
import uuid
import base64
import pytesseract
from IPython import display
from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pinecone import Pinecone
import numpy as np
from openai import OpenAI
from langchain.vectorstores import Pinecone as pine

In [2]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
from dotenv import load_dotenv
load_dotenv() 



openai_api_key = os.getenv("open_ai_key")

In [4]:
output_path = "./images"

In [5]:
# Get elements
raw_pdf_elements = partition_pdf(
    filename="D:/vera - Copy (2)/Samples/sample1.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    extract_image_block_output_dir=output_path,
)

In [6]:
# Categorize elements by type
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables

# Get text, tables
texts, tables = categorize_elements(raw_pdf_elements)

# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0
)
joined_texts = " ".join(texts)
texts_4k_token = text_splitter.split_text(joined_texts)

In [7]:
# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    model = ChatOpenAI(temperature=0, model="gpt-4o-mini",api_key = openai_api_key)
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries

# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

In [8]:
text_summaries[0]

'**Summary of SriLankan Airlines Annual Report 2023/24**\n\n- **Overview**: SriLankan Airlines, the national carrier of Sri Lanka, aims to be the preferred airline for travelers seeking an authentic Sri Lankan experience. The report covers the financial year from April 1, 2023, to March 31, 2024.\n\n- **Vision & Mission**: The airline aspires to promote Sri Lanka globally, support tourism, and develop an air transport hub while maintaining ethical business practices.\n\n- **Financial Highlights**: \n  - Group Revenue: LKR 339.6 billion\n  - Group Operating Profit (before exchange gains): LKR 39.7 billion\n  - Net Profit: LKR 7.9 billion\n  - Passenger Revenue: LKR 276.2 billion; Cargo Revenue: LKR 31 billion (39% decrease).\n\n- **Operational Challenges**: The airline faced labor shortages, fleet constraints, and increased operational costs due to global supply chain issues. Over 1,000 flights were canceled, costing over USD 60 million.\n\n- **Strategic Initiatives**: \n  - Wet leasing

In [ ]:
# Get image summaries
image_elements = []
image_summaries = []

def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

def summarize_image(encoded_image):
    prompt = [
        SystemMessage(content="You are financial analyst tasking with providing investment advice"),
        HumanMessage(content=[
            {
                "type": "text",
                "text": "You are financial analyst tasking with providing investment advice.\n"
            "You will be given a mixed of text, tables, and image(s) usually of charts or graphs.\n"
            "Use this information to provide investment advice related to the user question. \n"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            },
        ])
    ]
    response = ChatOpenAI(model="gpt-4o-mini-2024-07-18", openai_api_key=openai_api_key, max_tokens=1024).invoke(prompt)
    return response.content

for i in os.listdir(output_path):
    if i.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(output_path, i)
        encoded_image = encode_image(image_path)
        image_elements.append(encoded_image)
        summary = summarize_image(encoded_image)
        image_summaries.append(summary)


In [10]:
image_summaries[0]

"To provide investment advice regarding SriLankan Airlines, I would need specific data or context about the airline's current financial performance, market position, trends in the aviation industry, and any recent developments or challenges the airline is facing. Key factors to consider include:\n\n1. **Financial Performance**: Review revenue, profit margins, and operational costs. Look for trends in passenger numbers and cargo revenue.\n\n2. **Market Trends**: Analyze the travel demand, especially post-pandemic recovery, and how it impacts the airline industry.\n\n3. **Competitive Landscape**: Understand how SriLankan Airlines compares to competitors in terms of routes, pricing, and service quality.\n\n4. **Regulatory Environment**: Consider any government policies or regulations affecting the airline, especially in relation to international travel.\n\n5. **Strategic Initiatives**: Look into any strategic partnerships, fleet expansion plans, or sustainability initiatives.\n\n6. **Econ

**FAISS vectordb**

In [17]:
# Create Documents and Vectorstore
documents = []
retrieve_contents = []

for e, s in zip(texts, text_summaries):
    i = str(uuid.uuid4())
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'text',
            'original_content': e
        }
    )
    retrieve_contents.append((i, e))
    documents.append(doc)

for e, s in zip(tables, table_summaries):
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'table',
            'original_content': e
        }
    )
    retrieve_contents.append((i, e))
    documents.append(doc)

for e, s in zip(image_elements, image_summaries):
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'image',
            'original_content': e
        }
    )
    retrieve_contents.append((i, s))
    documents.append(doc)

vectorstore = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)


In [ ]:
db.docstore._dict

**As a single function - adding to existing database**

In [ ]:
def add_new_vectors_to_faiss(texts,text_summaries,tables,table_summaries,image_elements,image_summaries):
    # Create Documents and Vectorstore
    documents = []
    retrieve_contents = []

    for e, s in zip(texts, text_summaries):
        i = str(uuid.uuid4())
        doc = Document(
            page_content = s,
            metadata = {
                'id': i,
                'type': 'text',
                'original_content': e
            }
        )
        retrieve_contents.append((i, e))
        documents.append(doc)

    for e, s in zip(tables, table_summaries):
        doc = Document(
            page_content = s,
            metadata = {
                'id': i,
                'type': 'table',
                'original_content': e
            }
        )
        retrieve_contents.append((i, e))
        documents.append(doc)

    for e, s in zip(image_elements, image_summaries):
        doc = Document(
            page_content = s,
            metadata = {
                'id': i,
                'type': 'image',
                'original_content': e
            }
        )
        retrieve_contents.append((i, s))
        documents.append(doc)

    vectorstore = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))
    vectorstore.save_local("faiss_index_new")
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    db_new = FAISS.load_local("faiss_index", embeddings)
    db = FAISS.load_local("faiss_index", embeddings)
    db = db.merge_from(db_new)

    return db   

    

**PINECONE vector db**

In [ ]:
pc = Pinecone(api_key="c5198c4f-c653-4860-a900-cd2e67f21ce3")
index = pc.Index("multimodal")

In [ ]:
openai_client = OpenAI(
    api_key= openai_api_key
)
MODEL = "text-embedding-ada-002"


In [ ]:
def generate_embeddings_and_upsert(texts):
    batch_size = 10  # Adjust based on your rate limits and testing
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        # Generate embeddings for the batch
        res = openai_client.embeddings.create(
            input=batch_texts,
            model=MODEL
        )
        embeddings = [np.array(record.embedding) for record in res.data]
        
        # Upsert embeddings to Pinecone with metadata
        vectors = []
        for text, embedding in zip(batch_texts, embeddings):
            vectors.append({
                "id": str(uuid.uuid4()),  # Generate a unique ID for each vector
                "values": embedding.tolist(),  # Convert embedding to list
                "metadata": {"text": text}  # Include original text as metadata
            })
        index.upsert(vectors=vectors)
        
# Combine all summaries into one list
all_summaries = table_summaries + text_summaries + image_summaries

generate_embeddings_and_upsert(all_summaries)